In [711]:
# Importing Library

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,gradient_boosting
from sklearn.impute import KNNImputer,SimpleImputer
from sklearn.cluster import DBSCAN,k_means
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso,LassoCV,LinearRegression,Ridge,RidgeCV
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV,KFold,ParameterGrid,train_test_split
#from sklearn.neighbors import kd_tree,KDTree,KNeighborsClassifier
from sklearn.metrics import accuracy_score,auc,classification_report,confusion_matrix,euclidean_distances,euclidean_distances,f1_score
from sklearn.metrics import plot_roc_curve
from  sklearn.pipeline import Pipeline
from sklearn.svm import SVR,SVR
from sklearn.preprocessing import OneHotEncoder,power_transform,StandardScaler
from statsmodels.stats import anova,outliers_influence
import warnings
warnings.filterwarnings('ignore')

In [775]:
# loading the training  dataset
df_Train=pd.read_csv(r'D:\Datatrained\Practice Project Data Trained\Bigdata Mart\bigdatamart_Train.csv')
df_Train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [776]:
# loading the training  dataset

df_Test=pd.read_csv(r'D:\Datatrained\Practice Project Data Trained\Bigdata Mart\bigdatamart_Test.csv')
df_Test.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [777]:
# Analyzing the each colum whether they are categorialor continueous
pd.set_option('display.max_columns',25)

In [778]:
df_Train['source']='df_Train'
df_Test['source']='df_Test'

data = pd.concat([df_Train, df_Test],ignore_index=True)

print(df_Train.shape)
print(df_Test.shape)
print(data.shape)
    

(8523, 13)
(5681, 12)
(14204, 13)


In [779]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,df_Train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,df_Train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,df_Train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,df_Train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,df_Train


In [780]:
data.apply(lambda x: sum(x.isnull()))

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [781]:
#Looking on basic statistics for numerical variables.

In [782]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [783]:
# Missing Values in Item_Weight,Item_Outlet_Sales
#Zero value for Item visibility,High Std
# High Std for Item_MRP,huge gap between min and median,huge gap between min and max
#Outlet_Establishment_Year vary from 1985 to 2009,Lets look  at age of particular store,it might have better impact on sale

In [784]:
# Looking for Data type in dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

In [785]:
# Lets look on number of unique value for each of categorical variable

In [786]:
data.apply(lambda x: len(x.unique()))

Item_Identifier               1559
Item_Weight                    416
Item_Fat_Content                 5
Item_Visibility              13006
Item_Type                       16
Item_MRP                      8052
Outlet_Identifier               10
Outlet_Establishment_Year        9
Outlet_Size                      4
Outlet_Location_Type             3
Outlet_Type                      4
Item_Outlet_Sales             3494
source                           2
dtype: int64

In [787]:
# In dataset,we have 1599 item in 10 stores and 16 uniqure product type.

In [788]:
#Filter categorical variables
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
#Exclude ID cols and source:
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
#Print frequency of categories
for col in categorical_columns:
    print ('\nFrequency of Categories for varible %s'%col)
    print (data[col].value_counts())


Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

F

In [789]:
#In dataset,all the featurer are categoriacl and continueous in nature

In [790]:
data.shape
# There are 14204 rows and 13 colums

(14204, 13)

In [791]:
# Data Cleaning

In [792]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

In [793]:
data['Item_Weight']=data['Item_Weight'].fillna(data['Item_Weight'].mean())
data['Outlet_Size']=data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0])


In [794]:
data['Outlet_Size'].value_counts()

Medium    8671
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [795]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [796]:
# Null values are treated except Item_Outlet_Sales.Since We merged training and test data together

In [797]:
#Feature Engineering

In [798]:
#combining Outlet_Type

In [799]:
data.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,339.828500
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3694.038558


In [800]:
#significant difference between them and we’ll leave them as it is.

In [801]:
# Modify Item_Visibility with mean as it is continueous data

In [802]:
#data['Item_Visibility']=data['Item_Visibility'].fillna(data['Item_Visibility'].mean())


In [803]:
# Item_Weigh and Outlet_Size have no missing  value

In [804]:
#Determine average visibility of a product

data['Item_Visibility']=data['Item_Visibility'].fillna(data['Item_Visibility'].mean())


In [805]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [806]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [807]:
#Determine the years of operation of a store
#new column depicting the years of operation of a store.

In [808]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Type_Combined
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,df_Train,Food
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,df_Train,Drinks
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,df_Train,Food
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,Tier 3,Grocery Store,732.3800,df_Train,Food
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,df_Train,Non-Consumable


In [809]:
#Years:
data['Outlet_Years'] = 2020 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        22.169319
std          8.371664
min         11.000000
25%         16.000000
50%         21.000000
75%         33.000000
max         35.000000
Name: Outlet_Years, dtype: float64

In [810]:
#Modify categories of Item_Fat_Content

In [811]:
#Change categories of low fat:
print ('Original Categories:')
data['Item_Fat_Content'].value_counts()

print ('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
data['Item_Fat_Content'].value_counts()

Original Categories:

Modified Categories:


Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [812]:
# Numerical and One-Hot Coding of Categorical variables

In [813]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [814]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [815]:
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
Outlet_1                       uint8
O

In [816]:
data[['Item_Fat_Content_0','Item_Fat_Content_1','Outlet_Location_Type_2']].head(10)

,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_2
0,1,0,0
1,0,1,1
2,1,0,0
3,0,1,1
4,1,0,1
5,0,1,1
6,0,1,1
7,1,0,1
8,0,1,0
9,0,1,0


In [817]:
#Exporting Data

In [818]:
#Drop the columns which have been converted to different types:
x=data.drop(columns=['Item_Type','Outlet_Establishment_Year'],axis=1)

x1=x.drop(columns=['Item_Identifier','Outlet_Identifier'],axis=1)


#Export files as modified versions:


In [819]:
x1.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,source,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,...,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,3735.1380,df_Train,21,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,443.4228,df_Train,11,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,2097.2700,df_Train,21,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,732.3800,df_Train,22,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,994.7052,df_Train,33,1,0,0,0,1,1,...,0,1,0,1,0,0,0,0,0,0,0,0


In [820]:
#Divide into test and train:
train = x1.loc[x1['source']=="df_Train"]
test = x1.loc[x1['source']=="df_Test"]

In [821]:
train

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,source,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,...,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.300,0.016047,249.8092,3735.1380,df_Train,21,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1
1,5.920,0.019278,48.2692,443.4228,df_Train,11,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0
2,17.500,0.016760,141.6180,2097.2700,df_Train,21,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,1
3,19.200,0.000000,182.0950,732.3800,df_Train,22,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0,0,0
4,8.930,0.000000,53.8614,994.7052,df_Train,33,1,0,0,0,1,1,...,0,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,6.865,0.056783,214.5218,2778.3834,df_Train,33,1,0,0,0,1,1,...,1,0,0,1,0,0,0,0,0,0,0,0
8519,8.380,0.046982,108.1570,549.2850,df_Train,18,0,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1,0,0
8520,10.600,0.035186,85.1224,1193.1136,df_Train,16,1,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0,0,0
8521,7.210,0.145221,103.1332,1845.5976,df_Train,11,0,1,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0,0,0


In [822]:

#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1)
train.drop(['source'],axis=1,inplace=True)

In [823]:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [824]:
train.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,...,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,3735.1380,21,1,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,443.4228,11,0,1,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,2097.2700,21,1,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,732.3800,22,0,1,0,0,1,0,1,...,1,0,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,994.7052,33,1,0,0,0,1,1,0,...,0,1,0,1,0,0,0,0,0,0,0,0


In [825]:
y=train['Item_Outlet_Sales']
x=train.drop(columns=['Item_Outlet_Sales'])

In [826]:
x.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,...,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,21,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,11,0,1,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,21,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,22,0,1,0,0,1,0,1,0,...,1,0,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,33,1,0,0,0,1,1,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0


In [827]:
regression=LinearRegression()

In [828]:
# split data into train and test. Model will be built on training data and tested on test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=20)
y_train.head()

307      874.8612
2391    2024.0320
3552    5195.2374
3153    2316.9840
6313    1278.3360
Name: Item_Outlet_Sales, dtype: float64

In [829]:
regression.fit(x_train,y_train)

LinearRegression()

In [830]:
#Adjusted R2 SCORE
regression.score(x_train,y_train)

0.561256139381791

In [836]:
y_pred=regression.predict(x_test)

In [837]:
y_pred

array([1771.65731235, 3719.71027862,  898.24668909, ..., 2125.42554574,
       4219.22762577, 3938.23358115])

In [839]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [841]:
mean_absolute_error(y_test,y_pred)

822.7535816442023

In [842]:
np.sqrt(mean_squared_error(y_test,y_pred))

1109.5315930746867

LassoCV(normalize=True)

0.05643095546615427

Lasso(alpha=0.05643095546615427)

0.5694624012017097

In [854]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [855]:
# DecisionTreeRegressor

In [856]:


from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(x_train,y_train)
preddt=dt.predict(x_test)
print(mean_squared_error(y_test,preddt))
print(r2_score(y_test,preddt))
print(mean_absolute_error(y_test,preddt))
dt.score(x_test,y_test)

2391736.247823725
0.16359347225653575
1063.5859748474895


0.16359347225653575

In [857]:
#RandomForestRegressor

from sklearn.ensemble import RandomForestRegressor
RF=RandomForestRegressor()
RF.fit(x_train,y_train)
pred_RF=RF.predict(x_test)
print(mean_squared_error(y_test,pred_RF))
print(r2_score(y_test,pred_RF))
print(mean_absolute_error(y_test,pred_RF))
RF.score(x_test,y_test)

1201837.4754906225
0.5797092130448118
770.0594028822151


0.5797092130448118

In [858]:
from sklearn.model_selection import cross_val_score
scr=cross_val_score(regression,x,y,cv=5)
print("Cross Validation Score fo Linear Regression :",scr.mean())

Cross Validation Score fo Linear Regression : 0.5610959879125044


In [861]:
from sklearn.model_selection import cross_val_score
scr=cross_val_score(dt,x,y,cv=5)
print("Cross Validation Score fo dt Regression :",scr.mean())

Cross Validation Score fo dt Regression : 0.1983211115150974


In [862]:
from sklearn.model_selection import cross_val_score
scr=cross_val_score(RF,x,y,cv=5)
print("Cross Validation Score fo RF Regression :",scr.mean())

Cross Validation Score fo RF Regression : 0.5521483614715869


In [ ]:
      Model Accuracy      Cross Validation Score      Diff
LR     56                        56                    0
DT     16                        19                    3
RF     57                        55                    2


Since diff is zero for LR.So, LR is the best model 

In [ ]:
#Hyperparameter tuning

In [863]:
from sklearn.linear_model import Ridge,Lasso,RidgeCV,LassoCV

In [864]:
lassCV= LassoCV(alphas=None,max_iter=1000,normalize=True)
lassCV.fit(x_train,y_train)

LassoCV(normalize=True)

In [865]:
# best Alpha parameter (Alpha = learning rate)
alpha =lassCV.alpha_
alpha

0.05643095546615427

In [866]:
# now that we have best parameter,lets use lasso regression and see how well our data ftted befor
lasso_reg=Lasso(alpha)
lasso_reg.fit(x_train,y_train)

Lasso(alpha=0.05643095546615427)

In [867]:
lasso_reg.score(x_test,y_test)

0.5694624012017097

In [ ]:
# After hyper parameter tuning,accuracy is not improved

In [ ]:
#Lasso Regression Model:

In [ ]:
import pickle

# writing diffrent model files to file

# saving model to local system
filename='finalized_model.picle'
pickle.dump(regression,open(filename,'wb'))

